In [1]:
from copy import deepcopy
import warnings
import importlib
warnings.filterwarnings('ignore')

from matplotlib import pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from IPython import display

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, Matern
from sklearn.gaussian_process.kernels import ConstantKernel as C
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from scipy.interpolate import UnivariateSpline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import lightgbm as lgb
import scipy.sparse
from cylp.cy import CyClpSimplex
from cylp.py.modeling.CyLPModel import CyLPArray

import src.kmeans as kmeans

sns.set_style('darkgrid')

In [ ]:
def data_preparation():
    """
    Чтение данных
    """
    train_path = 'data/train.csv'
    test_path = 'data/test.csv'

    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)

    train = train[train['galaxy'] != 'NGC 5253']
    train = train.reset_index(drop=True)

    test_sorted = test.sort_values('galactic year')
    test_sorted_ = test_sorted[test_sorted.columns[2:-1]]
    years = np.unique(np.array(test_sorted['galactic year']))
    year_fin = years[-1]
    year_prev = years[-2]

    test['galactic year'] = test['galactic year'].replace(year_fin, year_prev)

    train_ = train.copy(deep=True)
    train_ = train_.drop('y', axis=1)
    train_ = pd.concat((train_, test))

    enc = LabelEncoder()
    train_['galaxy'] = enc.fit_transform(train_['galaxy'])
    train['galaxy'] = enc.transform(train['galaxy'])
    test['galaxy'] = enc.transform(test['galaxy'])

    gxs = np.array(train['galaxy'].drop_duplicates())

    train_ = train.copy(deep=True)
    train_ = train_.drop('y', axis=1)
    train_ = pd.concat((train_, test))

    gxs = np.array(train['galaxy'].drop_duplicates())

    enc_h = OneHotEncoder()
    train_mat = enc_h.fit_transform(train[['galaxy']]).toarray()
    test_mat = enc_h.transform(test[['galaxy']]).toarray()

    train_mat = pd.DataFrame(train_mat)
    test_mat = pd.DataFrame(test_mat)
    train = pd.concat((train, train_mat), axis=1)
    test = pd.concat((test, test_mat), axis=1)
    
    return train, test, train_

# Регрессия на кластерах + оптимизация

In [3]:
importlib.reload(kmeans)

def clustering(train, N_CLUSTERS, random_state):
    """
    Функция для кластеризации временных рядов
    """
    df_to_clust = train.pivot(index='galactic year',
                              columns='galaxy', values='y').transpose()
    df_to_clust_val = df_to_clust.values
    df_to_clust[df_to_clust.columns] = df_to_clust_val
    df_clust = df_to_clust.fillna(-9999)
    array_to_clust = df_clust.to_numpy()
    mask = array_to_clust != -9999
    mask = mask.astype(int)

    selection_rule = mask.sum(axis=1) > 5
    mask = mask[selection_rule]
    array_to_clust = array_to_clust[selection_rule]

    km = kmeans.KMeans(array_to_clust[:, 15:], mask[:, 15:],
                       N_CLUSTERS, resolve_empty='singleton')
    km.initialise(seed=random_state)
    km.cluster()
    res = km.clustering_results
    res = np.where(res == 1)[1]
    df_to_clust = df_to_clust[selection_rule]

    galaxy_clusters = dict()
    galaxies = list(df_to_clust.index)
    for galaxy, cluster in zip(galaxies, res):
        galaxy_clusters[galaxy] = cluster
    return galaxy_clusters, df_to_clust, res, array_to_clust


def smoothing(df_to_clust, res, plot=False, mean=False):
    """
    Функция для сглаживания сплайнами
    """
    
    est_list = dict()
    for i in np.unique(res):
        output_plot = df_to_clust[res==i].index
        output = df_to_clust[res==i].transpose().to_numpy()
        pred = np.nanmean(output, axis=1)
        pred_arr = np.nanmean(output[:5], axis=0)
        if mean:
            output /= pred_arr
        df1 = pd.DataFrame(pred)
        df2 = pd.DataFrame(pred)
        df1.fillna(method='ffill', inplace=True)
        df2.fillna(method='bfill', inplace=True)
        arr1 = df1[0].to_numpy()
        arr2 = df2[0].to_numpy()
        pred = (arr1 + arr2) / 2
        spl = UnivariateSpline(np.arange(len(pred)), pred, k=5, s=10)
        spl.set_smoothing_factor(0.1)
        est = spl(np.arange(len(pred)))
        est_list[i] = est
        if plot:
            fig, ax = plt.subplots(1, 2, figsize=(20,7))
            plt.title(f'cluster: {i}')
            for y_arr, label in zip(output.T, list(output_plot)):
                ax[0].plot(y_arr, label=label, marker='o')
            ax[0].legend()
            ax[1].plot(pred)
            ax[1].plot(est)
            plt.show()
    return est_list


def reg_lin(train_, test, array_to_clust, df_to_clust,
            est_list, galaxy_clusters, plot=False):
    """
    Регрессия внутри кластеров
    """
    array_to_clust[array_to_clust < -100] = np.nan

    years = train_['galactic year'].drop_duplicates().to_numpy()
    preds_dict = dict()

    for num, row in enumerate(array_to_clust):
        galaxy = df_to_clust.iloc[num].name
        cluster = galaxy_clusters[galaxy]
        est = est_list[cluster]
        row = row.reshape(-1, 1)
        est = est.reshape(-1, 1)
        df = np.hstack((est, row))
        df = pd.DataFrame(df)
        df.columns = ['x', 'y']
        df_train = df.dropna()
        train_X = df_train[['x']]
        train_y = df_train['y']
        reg = GaussianProcessRegressor(kernel=DotProduct() + WhiteKernel())
        reg.fit(train_X, train_y)
        preds = reg.predict(df[['x']])
        preds = pd.DataFrame(preds).set_index(years)
        preds_dict[galaxy] = dict(zip(preds.index, preds.values))
        if plot:
            plt.figure(figsize=(16,4))
            plt.title(f'cluster: {cluster}')
            plt.plot(preds.to_numpy())
            plt.plot(df['y'].to_numpy())
            plt.show()
            plt.close()

    preds_list = list()

    for row_num in range(len(test)):
        row = test.iloc[row_num]
        galaxy = row['galaxy']
        year = row['galactic year']
        prediction = preds_dict[galaxy][year][0]
        preds_list.append(prediction)
    return preds_list, preds_dict


def regression(train, regressor, preds_dict, info=False):
    """
    Регрессия - один раунд кросс-валидации
    train содержит y
    """
    preds_list_train = list()

    for row_num in range(len(train)):
        row = train.iloc[row_num]
        galaxy = row['galaxy']
        year = row['galactic year']
        prediction = preds_dict[galaxy][year][0]
        preds_list_train.append(prediction)

    train_extended = train.copy(deep=True)
    train_extended['spline'] = preds_list_train

    train_extended = train_extended.fillna(-9999)

    X = train_extended.drop('y', axis=1)
    y = train_extended['y']

    reg = regressor
    reg.fit(X, y)
    preds = reg.predict(X)

    if info:
        plt.figure(figsize=(16,4))
        plt.plot(preds_list_train)
        plt.plot(train['y'].to_numpy())
        print(mean_squared_error(preds_list_train, train['y'].to_numpy()) ** 0.5)
        print(mean_squared_error(preds, train['y'].to_numpy()) ** 0.5)
    return regressor


def preds_lists(X_train, X_test, train_, N_CLUSTERS=20, random_state=7):
    """
    Функция для получения предикшенов по данным кластеризации
    """
    galaxy_clusters, df_to_clust, res, array_to_clust = clustering(X_train,
                                                                   N_CLUSTERS, random_state)

    est_list = smoothing(df_to_clust, res, plot=False, mean=False)
    preds_list, preds_dict = reg_lin(train_, X_test, array_to_clust,
                                     df_to_clust, est_list, galaxy_clusters, plot=False)
    train_preds_list, train_preds_dict = reg_lin(train_, X_train, array_to_clust,
                                     df_to_clust, est_list, galaxy_clusters, plot=False)
    test_preds_list, train_preds_dict = reg_lin(train_, test, array_to_clust,
                                     df_to_clust, est_list, galaxy_clusters, plot=False)
    return preds_list, train_preds_list, test_preds_list


def cluster_res(train, test, train_, random_state):
    """
    Обобщение результатов кластеризации
    """
    train_base = train[train['galactic year'] < test['galactic year'].min()]
    train_to_split = train[train['galactic year'] >= test['galactic year'].min()]
    X_train, X_test = train_test_split(train_to_split, test_size=0.00004, random_state=6, shuffle=True)
    X_train = pd.concat((train_base, X_train))

    preds_clust, train_preds_clust, test_preds_clust = [], [], []
    preds_list, train_preds_list, test_preds_list = preds_lists(X_train, X_test, train_,
                                                                N_CLUSTERS=20, random_state=random_state)
    preds_clust.append(preds_list)
    train_preds_clust.append(train_preds_list)
    test_preds_clust.append(test_preds_list)
    return preds_clust, train_preds_clust, test_preds_clust, X_train, X_test


def prediction(preds_clust, train_preds_clust, test_preds_clust, X_train, X_test, test):

    test_for_pred = test.copy(deep=True)
    spline_test_pred = pd.DataFrame(test_preds_clust).transpose()
    spline_test_pred.columns = [str(i) + '_spline' for i in spline_test_pred.columns]
    test_for_pred = test_for_pred.join(spline_test_pred)
    test_for_pred_ = test_for_pred.fillna(0)

    y_test = X_test['y'].copy(deep=True)
    test_X = X_test.reset_index(drop=True).copy(deep=True)
    spline_test = pd.DataFrame(preds_clust).transpose()
    spline_test.columns = [str(i) + '_spline' for i in spline_test.columns]
    test_X = test_X.drop('y', axis=1)
    test_X = test_X.fillna(-10).reset_index(drop=True)
    test_X = test_X.join(spline_test)

    train_X = X_train.copy(deep=True)
    spline_train = pd.DataFrame(train_preds_clust).transpose()
    spline_train.columns = [str(i) + '_spline' for i in spline_train.columns]
    train_y = train_X['y']
    train_X = train_X.drop('y', axis=1)
    train_X = train_X.fillna(0).reset_index(drop=True)
    train_X = train_X.join(spline_train)
    test_y = y_test

    sc = StandardScaler()
    train_X = sc.fit_transform(train_X)
    test_X = sc.transform(test_X)
    test_for_pred = sc.transform(test_for_pred_)

    regressor_boost = lgb.LGBMRegressor(num_leaves=20,
                                        learning_rate=0.05,
                                        n_estimators=200000,
                                        verbose=0)
    regressor_boost.fit(train_X, train_y, eval_set=[(test_X, test_y)],
                        eval_metric='l2', early_stopping_rounds=10, verbose=0)

    preds_boost = regressor_boost.predict(test_X)
    preds_on_test_boost_mult = regressor_boost.predict(test_for_pred)
    
    return preds_on_test_boost_mult

In [4]:
def optimize(preds, test, upper_bound_=None, lower_bound_=None):
    """
    Функция для выполнения оптимизации
    """

    preds_list = preds

    constr_matrix_1 = test['existence expectancy index'] < 0.7
    constr_matrix_1 = constr_matrix_1.to_numpy().astype(int).reshape(1, -1) * (-1)
    constr_matrix_1 = np.vstack((
        constr_matrix_1, np.ones(constr_matrix_1.size).reshape(1, -1)))
    constr_matrix_1 = scipy.sparse.csr_matrix(constr_matrix_1)
    constr_vector_1 = np.array([-5000, 50000])
    constr_matrix_2 = scipy.sparse.eye(test.shape[0])
    constr_vector_2 = np.ones(test.shape[0]) * 100
    lower_bound = np.zeros(test.shape[0])

    output = preds_list

    solver = CyClpSimplex()
    x_var = solver.addVariable('x', len(preds_list))

    vec1 = CyLPArray(constr_vector_1)
    solver.addConstraint(constr_matrix_1 * x_var <= vec1)
    vec2 = CyLPArray(constr_vector_2)
    solver.addConstraint(constr_matrix_2 * x_var <= vec2)
    if lower_bound_ is not None:
        solver.variablesLower = lower_bound_
    else:
        solver.variablesLower = lower_bound
    if upper_bound_ is not None:
        solver.variablesUpper = upper_bound_
    objective = CyLPArray(output - 1)
    solver.objective = objective * x_var
    solver.primal()
    out = solver.primalVariableSolution['x']

    std = np.ones(preds_list.size) * 0.0093

    output_matrix = np.repeat([preds_list], repeats=10000, axis=0)
    for i in range(output_matrix.shape[1]):
        np.random.seed(i)
        vec = np.random.normal(loc=0, scale=std[i], size=output_matrix.shape[0])
        output_matrix[:, i] = output_matrix[:, i] + vec

    optimized_matrix = np.zeros(output_matrix.shape)

    for row in tqdm(range(len(output_matrix))):
        objective = CyLPArray(output_matrix[row] - 1)
        solver.objective = objective * x_var
        solver.primal()
        optimized_matrix[row] = solver.primalVariableSolution['x']
    opt_pred = optimized_matrix.mean(axis=0)
    
    return opt_pred

In [5]:
train, test, train_ = data_preparation()

preds_gauss = list()
preds_boost = list()

for random_state in [7, 14]:

    preds_clust, train_preds_clust, test_preds_clust, X_train, X_test =\
        cluster_res(train, test, train_, random_state)

    preds_on_test_boost_mult =\
        prediction(preds_clust, train_preds_clust, test_preds_clust, X_train, X_test, test)
    
    preds_boost.append(preds_on_test_boost_mult)

preds_boost_first = np.array(preds_boost).mean(axis=0)

display.clear_output()

opt_pred = optimize(preds_boost_first, test)
opt_pred[0] -= 0.0001

# Вторая регрессия

In [6]:
def reg2(train, test, train_):
    train_base = train[train['galactic year'] < test['galactic year'].min()]
    train_to_split = train[train['galactic year'] >= test['galactic year'].min()]
    X_train, X_test = train_test_split(train_to_split, test_size=0.0004, random_state=6, shuffle=True)
    X_train = pd.concat((train_base, X_train))

    random_states = [7, 14]

    preds_clust, train_preds_clust, test_preds_clust = [], [], []
    for random_state in random_states:
        preds_list, train_preds_list, test_preds_list = preds_lists(X_train, X_test, train_,
                                                                    N_CLUSTERS=25, random_state=random_state)
        preds_clust.append(preds_list)
        train_preds_clust.append(train_preds_list)
        test_preds_clust.append(test_preds_list)
    return preds_clust, train_preds_clust, test_preds_clust, X_train, X_test

In [7]:
def add_features(train, test, train_):

    # Делаем новые фичи
    cols_to_use = train_.columns[2:]
    new_feature_matrix = train_.copy(deep=True)
    new_feature_matrix = new_feature_matrix.set_index('galaxy', drop=True)

    new_features = list()

    for column in cols_to_use:
        df = new_feature_matrix[[column
            ]].groupby(new_feature_matrix.index).agg([np.nanmean, np.nanmax, np.nanmin, np.nanstd])
        df.columns = [i + '_' + column for i in ['nanmean', 'nanmax', 'nanmin', 'nanstd']]
        new_features.append(deepcopy(df))
    new_features = pd.concat(new_features, axis=1)

    # Делаем агрегацию для таргета и вычисляем разницу первых и последних значений
    df_to_clust = train.pivot(index='galactic year',
                             columns='galaxy', values='y')
    df_to_clust = df_to_clust.to_numpy()
    diff = np.nanmean(df_to_clust[:10], axis=0) - np.nanmean(df_to_clust[-10:], axis=0)

    new_features['diff'] = diff
    new_features_ = new_features.dropna().copy(deep=True)
    transformations = dict()

    counter = 0
    for feature in new_features_.columns:
        cc = np.corrcoef(new_features_[feature], new_features_['diff'])
        cc_sqr = np.corrcoef(new_features_[feature] ** 2, new_features_['diff'])
        cc_inv = np.corrcoef(1 / new_features_[feature], new_features_['diff'])
        cc_ = np.max([np.abs(cc[0][1]), np.abs(cc_sqr[0][1]), np.abs(cc_inv[0][1])])
        tp = np.argmax([np.abs(cc[0][1]), np.abs(cc_sqr[0][1]), np.abs(cc_inv[0][1])])
        transformation = [lambda x: x, lambda x: np.square(x), lambda x: 1 / x, lambda x: 1 / np.square(x)][tp]
        transf_type = ['None', 'Square', 'Inversion', 'Quad inversion'][tp]
        if cc_ > 0.6 and feature != 'diff':
            transformations[feature] = transformation

    new_features = new_features[transformations.keys()]
    for key, value in transformations.items():
        new_features[key] = new_features[key].apply(value)


    train = train.join(new_features, on='galaxy')
    test = test.join(new_features, on='galaxy')
    
    return train, test


def fillna_y_train(train):
    """
    Новый признак для обучения
    """
    y = pd.pivot_table(train, values=['y'], index='galaxy', columns='galactic year').transpose()
    y_prev = y.to_numpy()
    y_transformed = y_prev.copy()

    kf = KFold(n_splits=7, shuffle=True, random_state=10)

    nan_idxs_list = list()

    idxs_list = np.indices(y_prev.shape)
    idxs_list = idxs_list.reshape(-1, np.product(y_prev.shape)).T

    splits = list(kf.split(idxs_list))

    for split in tqdm(splits):
        y_to_mask = deepcopy(y_prev)
        nan_idxs = idxs_list[split[1]]
        y_to_mask[nan_idxs[:, 0], nan_idxs[:, 1]] = np.nan
        imp = IterativeImputer(missing_values=np.nan, max_iter=40)
        y_out = imp.fit_transform(y_to_mask)
        y_transformed[nan_idxs[:, 0], nan_idxs[:, 1]] = y_out[nan_idxs[:, 0], nan_idxs[:, 1]].copy()

    y_out = pd.DataFrame(y_transformed, columns=y.columns.astype(str))
    y_out['galactic year'] = y.index.get_level_values(1)
    y_out = pd.melt(y_out, id_vars=['galactic year'], value_vars=list(y_out.columns[:-1]))
    y_out['galaxy'] = y_out['galaxy'].astype(int)
    y_out_masked = y_out.set_index(['galaxy','galactic year']).copy(deep=True)
    
    return y_out_masked


def fillna_y_final(train):
    """
    Новый признак для предсказания
    """
    y = pd.pivot_table(train, values=['y'], index='galaxy', columns='galactic year').transpose()
    y_prev = y.to_numpy()
    estimator = GaussianProcessRegressor(kernel=DotProduct() + WhiteKernel())
    imp = IterativeImputer(estimator=estimator, missing_values=np.nan, max_iter=40)
    y_out = imp.fit_transform(y_prev)
    y_out = pd.DataFrame(y_out, columns=y.columns.astype(str))
    y_out['galactic year'] = y.index.get_level_values(1)
    y_out = pd.melt(y_out, id_vars=['galactic year'], value_vars=list(y_out.columns[:-1]))
    y_out['galaxy'] = y_out['galaxy'].astype(int)
    y_out_final = y_out.set_index(['galaxy','galactic year']).copy(deep=True)
    
    return y_out_final


def regression_second(train, test, y_out_masked, y_out_final):

    X_train, X_test = train_test_split(train, test_size=0.00003, random_state=7, shuffle=True)

    test_for_pred = test.copy(deep=True)
    test_for_pred_ = test_for_pred.fillna(0)
    test_for_pred_ = test_for_pred_.join(y_out_final, on=['galaxy','galactic year'], how='left')

    preds_rnd = deepcopy(y_out_masked)
    n_out = preds_rnd.shape[0]
    np.random.seed(1)
    rnd_vec = np.random.normal(loc=0, scale=0.000002, size=n_out)

    y_test = X_test['y'].copy(deep=True)
    test_X = X_test.reset_index(drop=True).copy(deep=True)
    test_X = test_X.drop('y', axis=1)
    test_X = test_X.fillna(0).reset_index(drop=True)
    test_X = test_X.join(y_out_masked, on=['galaxy','galactic year'], how='left')

    train_X = X_train.copy(deep=True)
    train_y = train_X['y']
    train_X = train_X.drop('y', axis=1)
    train_X = train_X.fillna(0).reset_index(drop=True)
    train_X = train_X.join(preds_rnd, on=['galaxy','galactic year'], how='left')
    test_y = y_test

    sc = StandardScaler()
    train_X = sc.fit_transform(train_X)
    test_X = sc.transform(test_X)
    test_for_pred = sc.transform(test_for_pred_)

    regressor_gp = GaussianProcessRegressor(kernel=Matern() * C() 
                                            + WhiteKernel()
                                            + DotProduct() * C())
    regressor_gp.fit(train_X, train_y)
    preds_gp = regressor_gp.predict(test_X)

    regressor_boost = lgb.LGBMRegressor(num_leaves=27,
                                        learning_rate=0.05,
                                        n_estimators=5000,
                                        verbose=0)
    regressor_boost.fit(train_X, train_y,
                        eval_set=[(test_X, test_y)],
                        eval_metric='l2',
                        early_stopping_rounds=10,
                        verbose=0)

    preds_boost = regressor_boost.predict(test_for_pred)
    preds_gp = regressor_gp.predict(test_for_pred)
    
    return preds_boost, preds_gp

In [8]:
train, test, train_ = data_preparation()

preds_clust, train_preds_clust, test_preds_clust, X_train, X_test = reg2(train, test, train_)
preds_boost_first =\
        prediction(preds_clust, train_preds_clust, test_preds_clust, X_train, X_test, test)

train, test, train_ = data_preparation()
train, test = add_features(train, test, train_)
y_out_masked = fillna_y_train(train)
y_out_final = fillna_y_final(train)

preds_boost_second, preds_gp_second = regression_second(train, test, y_out_masked, y_out_final)

pred = 0.29 * preds_boost_first + 0.29 * preds_gp_second + 0.42 * preds_boost_second

display.clear_output()

In [16]:
output = pd.read_csv('data/sample_submit.csv')
output['pred'] = pred
output['opt_pred'] = opt_pred
output = output.set_index('index')
output.to_csv(f'output.csv')